# Google search try

In [ ]:
from googlesearch import search
import re
import pandas as pd

In [ ]:
companies = pd.read_csv('../data/processed/companies.csv')

In [ ]:
companies = companies['name'].tolist()

In [ ]:
# to search
found = set()

for idx,company in enumerate(companies):
    for j in search(company, tld="com"):
        print(idx)
        if re.search('wikipedia',j):
            found.add(company)
            break

In [ ]:
search

In [ ]:
import bs4 as bs
import urllib

comp = 'The Bank of East Asia Ltd'
url = 'https://en.wikipedia.org/wiki/'
print(urllib.parse.urlencode(comp))
# source = urllib.request.urlopen(url).read()

In [ ]:
soup = bs.BeautifulSoup(source,'html')

In [ ]:
for tag in soup.find_all('td'):
    if tag.get('class') == ['infobox-data', 'adr']:
        print(tag.text)

# Nominatim

In [1]:
from geopy.geocoders import Nominatim
import pandas as pd 
import os
import sys
import ssl
import numpy as np


sys.path.append('../')

In [2]:
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

In [3]:
geolocator = Nominatim(user_agent="my-application3", ssl_context=ssl_context, timeout=None)

In [4]:
# Random test
location = geolocator.geocode("175 5th Avenue NYC, United states")
location.latitude 

40.741059199999995

In [5]:
address = pd.read_parquet('../data/raw/addresses.parquet.gzip')
country_codes  = pd.read_excel('../data/raw/Countries.xlsx')

In [6]:
address.head()

,id,name,country,city_state_postal,location_street1
0,50472,"Infinity Pharmaceuticals, Inc.",USA,"Cambridge, MA 02138",1100 Massachusetts Avenue
1,282940,"Bozhon Precision Industry Technology Co., Ltd.",CHN,"Suzhou, Jiangsu 215222",666 Huxin West Road
2,313953,Quaker Houghton,USA,"Conshohocken, PA 19428-2380",One Quaker Park
3,1441,Kohl's Corp.,USA,"Menomonee Falls, WI 53051",N56 West17000 Ridgewood Drive
4,135666,"Sorrento Therapeutics, Inc.",USA,"San Diego, CA 92121",4955 Directors Place


In [7]:
address['completed'] = np.nan 

In [8]:
address['city'] = address['city_state_postal'].str.split(',').str[0]

In [9]:
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Aachen'), 'country'] = 'DEU'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Artarmon'), 'country'] = 'AUS'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Ann Arbor'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Bangalore'), 'country'] = 'IND'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Blindern'), 'country'] = 'NOR'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Boulder'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Cambridge'), 'country'] = 'GBR'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Lausanne'), 'country'] = 'CHE'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Milwaukee'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Munich'), 'country'] = 'DEU'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'New York'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Norwich'), 'country'] = 'GBR'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Oberhaching'), 'country'] = 'DEU'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Paris'), 'country'] = 'FRA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Philadelphia'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Snoqualmie'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Southborough'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Stockholm 111 37'), 'country'] = 'SWE'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Towson'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Waltham'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Zurich 8002'), 'country'] = 'CHE'

# Adding the only other company with location_street1
address.loc[(address['name'] == 'Fennoa Oy'), 'city'] = 'Alavus'
address.loc[(address['name'] == 'Fennoa Oy'), 'country'] = 'FIN'
address.loc[(address['name'] == 'Fennoa Oy'), 'city_state_postal'] = 'Alavus, 63300'
address.loc[(address['name'] == 'Fennoa Oy'), 'location_street1'] = 'Mikko Kalliovaara Rinteentie 136'

In [ ]:
address = address.merge(country_codes, how='left', left_on='country', right_on='Alpha-3 code')

In [ ]:
address

In [ ]:
address_new = address.loc[(address['completed'] != 1)&(address['country'] != 'NAM'))]

for id, rows in address_new.iterrows():

    print(id)

    if rows['completed'] != 1:

        if rows['city_state_postal'] is None:
            continue

        else:
             
            i = 0
            while i < len(rows['city_state_postal'].split(',')) :
                search_str = rows['city_state_postal'].split(',')[0:i].join(',') + rows['country']
                x = geolocator.geocode(search_str)
                if x is None:
                    i += 1
                    continue
                if x is not None:
                    lat = x.latitude
                    log = x.longitude
                    address.loc[id,'lat'] = lat
                    address.loc[id,'log'] = log
                    address.loc[id,'completed'] = 1
                    break
    
    if id % 1000 == 0:
        address.to_parquet('../data/processed/addresses.parquet.gzip', compression='gzip')
        print('saved')

In [ ]:
address = pd.read_parquet('../data/processed/addresses.parquet.gzip')

In [ ]:
address.to_parquet('../data/processed/addresses.parquet.gzip', compression='gzip')

In [ ]:
address.country.isna().sum()

In [ ]:
country_codes 

In [ ]:
uni = address.groupby('city').agg(
    unique_country = pd.NamedAgg('country',lambda x: x.unique().tolist())
).reset_index()

In [ ]:
mult = uni.loc[uni['unique_country'].apply(lambda x: len(x)) > 1]

In [ ]:
mult_city = mult.explode('unique_country')

In [ ]:
address = mult_city.merge(country_codes, left_on='unique_country', right_on='Alpha-3 code', how='left')

In [ ]:
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Aachen'), 'Country'] = 'Germany'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Artarmon'), 'Country'] = 'Australia'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Ann Arbor'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Bangalore'), 'Country'] = 'India'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Boulder'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Cambridge'), 'Country'] = 'United Kingdom'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Lausanne'), 'Country'] = 'Switzerland'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Milwaukee'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Munich'), 'Country'] = 'Germany'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'New York'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Norwich'), 'Country'] = 'United Kingdom'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Oberhaching'), 'Country'] = 'Germany'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Paris'), 'Country'] = 'France'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Philadelphia'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Snoqualmie'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Southborough'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Stockholm 111 37'), 'Country'] = 'Sweden'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Towson'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Waltham'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Zurich 8002'), 'Country'] = 'Switzerland'

In [ ]:
address

In [ ]:
for id, row in address.iterrows():
    print(id)
    mask = (address['city'] == row['city'])&(address['country'] == row['unique_country'])

    x = geolocator.geocode(row['city']+ ", "+ row['Country'])
    
    if x is not None:

        lat = x.latitude
        log = x.longitude

        address.loc[mask,'lat'] = lat
        address.loc[mask,'log'] = log

    if x is None:

        address.loc[mask, 'lat'] = None
        address.loc[mask, 'log'] = None
    
    if id % 100 == 0:
        address.to_parquet('../data/processed/addresses.parquet.gzip', compression='gzip')
        print('saved')
    

In [ ]:
address.to_csv('../data/processed/addresses.csv')